# HA4: Bitcoin
Update: the new deadline is December, 21.

The preferable format of the solution is IPython notebook.

## Pre-requisite:
1. Generate 3 testnet addresses (address_1, address_2, address_3; should start with either m or n)

2. From prerequisite 1,  generate 2-3 MultiSignature address (2 e.g. private_keys for {addess_1, address_2, address_3} out of three are required to spend Bitcoins). Note that your 2-3 MultiSignature address will start with 2. You may find https://www.youtube.com/watch?v=v8xKz-JPSOs&list=PLIRHcs7vupAvd18RbDJ0OYe-f_VME9H53&index=21 useful.

3. If you face any problems with bitcoin library, try

Install version 1.1.39:
!pip uninstall bitcoin

!pip install bitcoin==1.1.39

Or replace bitcoin.* with bitcoin.transaction.* For example,
bitcoin.mk_multisig_script(pub_keys,m)-> bitcoin.transaction.mk_multisig_script(pub_keys,m)

Or use bitcoinlib (https://github.com/1200wd/bitcoinlib)

In [1]:
import hashlib
import os
import hashlib
import binascii

In [2]:
pip install base58 bitcoin ecdsa

Note: you may need to restart the kernel to use updated packages.


In [3]:
import base58
import ecdsa
import bitcoin
from ecdsa import SigningKey, SECP256k1

In [4]:
pip install ecdsa

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install PrettyTable

Note: you may need to restart the kernel to use updated packages.


In [6]:
from prettytable import PrettyTable

In [7]:
def hashing(a):
    first_sha256 = hashlib.sha256(a)
    ripemd160 = hashlib.new("ripemd160")
    ripemd160.update(first_sha256.digest())
    return_0 = ripemd160.digest()
    return_1 = bytes.fromhex("00") + ripemd160.digest()
    return_2 = bytes.fromhex("6f") + ripemd160.digest()
    return return_0, return_1, return_2

def checksum(b):
    checksum_full = hashlib.sha256(hashlib.sha256(b).digest()).digest()
    new_checksum = checksum_full[:4]
    return b + new_checksum

def to_base58(c):
    return base58.b58encode(c).decode('utf-8')

def compressed_key(d):
    a = d.hex()

    if a[-1] == '0' or a[-1] == '2' or a[-1] == '4' or a[-1] == '6' or a[-1] == '8' or a[-1] == 'a' or a[-1] == 'c' or a[-1] == 'e':
        return bytes.fromhex("02") + d
    
    else:
        return bytes.fromhex("03") + d
    
def generate_address(print_output = True):
    
    private_key = os.urandom(32).hex()
    private_key_from_hex = bytes.fromhex(private_key)

    import ecdsa
    sk = ecdsa.SigningKey.from_string(private_key_from_hex, curve = ecdsa.SECP256k1) 


    verification_key = sk.verifying_key
    public_key = bytes.fromhex("04") + verification_key.to_string()
    compressed_public_key = compressed_key(verification_key.to_string())
    compressed_public_key_hex = compressed_public_key.hex()[:66]
    decoded_pubkey, mainnet_pubkey, testnet_pubkey = hashing(public_key)

    checksum_main_pubkey = checksum(mainnet_pubkey)
    checksum_test_pubkey = checksum(testnet_pubkey)


    main_address = to_base58(checksum_main_pubkey)
    test_address = to_base58(checksum_test_pubkey)


    if print_output:
        x = PrettyTable()
        x.field_names = ["Discription", "Value"]
        x.add_row(["Private Key, hex", private_key])
        x.add_row(["Public Key, hex", public_key.hex()])
        x.add_row(["Compressed Public Key, hex", compressed_public_key_hex])
        x.add_row(["MainNet Address, base58", main_address])
        x.add_row(["Testnet Address, base58", test_address])
        print(x)
    return(private_key, public_key.hex(), compressed_public_key_hex, main_address, test_address)

print('Data from the first address')
address_data_1 = generate_address()
print('Data from the second address')
address_data_2 = generate_address()
print('Data from the third address')
address_data_3 = generate_address()

Data from the first address
+----------------------------+------------------------------------------------------------------------------------------------------------------------------------+
|        Discription         |                                                               Value                                                                |
+----------------------------+------------------------------------------------------------------------------------------------------------------------------------+
|      Private Key, hex      |                                  1acc20597b1a487b1578ed4e08092644c28c55f8f21794bcba7942ceff88187f                                  |
|      Public Key, hex       | 04a92d69cdeedc1d359ce623fc2f5240a341882b0cc0d2cc03804fcebce2ecf3ae8d9b125cfe4f4eb79bf94b94c264e95066db9b103c4418c9860f3131f2f1c1c3 |
| Compressed Public Key, hex |                                 03a92d69cdeedc1d359ce623fc2f5240a341882b0cc0d2cc03804fcebce2ecf3ae                       

In [9]:
def public_to_test(a):
    decoded_address = base58.b58decode(scriptaddress)
    address_1 = decoded_address[1:-4]
    hex_address = bytes.fromhex("C4") + address_1
    checksum_test_multsig = checksum(hex_address)
    test_address = to_base58(checksum_test_multsig)
    return test_address
    
def checksum(b):
    checksum_full = hashlib.sha256(hashlib.sha256(b).digest()).digest()
    new_checksum = checksum_full[:4]
    return b + new_checksum

def to_base58(c):
    return base58.b58encode(c).decode('utf-8')

a = 2
b = 3
pub_keys = [address_data_1[2], address_data_2[2], address_data_3[2]]

    
rawscript = bitcoin.mk_multisig_script(pub_keys,a)
descript = bitcoin.deserialize_script(rawscript)
scriptaddress = bitcoin.scriptaddr(rawscript)
print("MultiSig Address: \n", scriptaddress)

multisig_adress = public_to_test(scriptaddress)
print("MultiSig Testnet Address: \n", multisig_adress)

print("\nRedeem Script: \n", rawscript)

MultiSig Address: 
 3DQt2sAPSpiNs2Sjk5kk1DBgz6FZ3M6VeV
MultiSig Testnet Address: 
 2N4y66c6R4HDj4p5HRDNcdAAxCSTiurc825

Redeem Script: 
 522103a92d69cdeedc1d359ce623fc2f5240a341882b0cc0d2cc03804fcebce2ecf3ae2102a5aa6ac17359ebeeb3843c24ffca8b105f258c8fc3183a6002d462456c2460122102a36f384e5625081d055e0c8f202571abb482ce8bc7369687857a1266cbcaf46253ae


## 1 Task (1 point) 
Calculate double Hash_256 of your name. Take two inputs from the user, i.e., first name and last name. Concatenate string and take SHA256(SHA256(<first name> <lastname>)).

In [10]:
from hashlib import sha256
my_name = "Karpunkin Yaroslav"
hashed = sha256(bytes(my_name,encoding='utf-8'))
print(f"After the 1 Hashing: \n{hashed.hexdigest()}")
my_name_double_hashed = sha256(hashed.digest()).hexdigest()
print(f"After the 2 Hashing: \n{my_name_double_hashed}")

After the 1 Hashing: 
a5cf8ff2a238f6a087727d890e093b076679643e60436e27dcc0dc107240da3b
After the 2 Hashing: 
f2f27716ab6c760a48a83e03474a1bb250e8814c51386f65e1126d55bba28cf5


## Task2 (1 point) 
Get some testnet Bitcoins to your testnet address_1 from any source. (For example: https://testnet-faucet.mempool.co/, https://coinfaucet.eu/en/btc-testnet/ or search "bitcoin testnet faucet"). Provide the Transaction_ID.

In [11]:
address_data_1[-1]

'mfk8F4iJamC36FQM8Y66Y8g88ENQcBAqJL'

We sent 0.01077657 bitcoins to address  
mfk8F4iJamC36FQM8Y66Y8g88ENQcBAqJL  

tx: 0514812bac079243b65aeb7991bc8446525a331993e760842171e5b00c28d9b0  
Send coins back, when you don't need them anymore to the address  

mv4rnyY3Su5gjcDNzbMLKBQkBicCtHUtFB  

Here is Bitcoin Testnet Transaction:  
https://live.blockcypher.com/btc-testnet/tx/0514812bac079243b65aeb7991bc8446525a331993e760842171e5b00c28d9b0/

In [1]:
multisig_adress = "mfk8F4iJamC36FQM8Y66Y8g88ENQcBAqJL"
multisig_redeem_script = "522103a92d69cdeedc1d359ce623fc2f5240a341882b0cc0d2cc03804fcebce2ecf3ae2102a5aa6ac17359ebeeb3843c24ffca8b105f258c8fc3183a6002d462456c2460122102a36f384e5625081d055e0c8f202571abb482ce8bc7369687857a1266cbcaf46253ae"

tx = "0514812bac079243b65aeb7991bc8446525a331993e760842171e5b00c28d9b0"
output_idx = 1
output_script = "027db649a3b298bca1c6de79f5ba590e72fd40ac"
# Первые 6 и последние 4
output_script_full = "76a914027db649a3b298bca1c6de79f5ba590e72fd40ac88ac"
my_name_double_hashed = "f2f27716ab6c760a48a83e03474a1bb250e8814c51386f65e1126d55bba28cf5"

## Task 3 (2 points)
Create a new transaction from address_1 and provide Transaction_ID. Make 2 outputs:

Output_1 : Some bitcoins to address_2.

Output_2: Data transfer with first 4 Bytes double Hash_256 of your name. You send 0 bitcoins in this output.

Note. Be aware of small transaction fees. You may find https://blockstream.info/testnet/tx/push useful.

In [2]:
def bytewise_reverse(hex_string):
    i = 0
    reverted_string = []
    while i != len(hex_string):
        temp_value = hex_string[i] + hex_string[i+1]
        reverted_string.append(temp_value)
        i = i + 2
    reverted_string.reverse()
    return ''.join(reverted_string)

def doublehash256_txid(a): 
    temp_bin = binascii.unhexlify(a)
    temp_hash = hashlib.sha256(temp_bin).digest()
    hash2 = hashlib.sha256(temp_hash).digest()
    return str(binascii.hexlify(hash2),"ascii")

def bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = bytewise_reverse(str(temp_1).zfill(16))
    return temp_2

def tx_inputs_tx_id(previous_tx_hash, previous_output_index, previous_output_script):
    previous_tx_hash_reversed = bytewise_reverse(previous_tx_hash)
    previous_output_index = bytewise_reverse('{:08x}'.format(int(previous_output_index)))
    return previous_tx_hash_reversed + previous_output_index + place_holder(previous_output_script) +  'ffffffff'

def place_holder(output_script):
    size_receiver_1 = hex(int(len(output_script)/2))[2:]
    script_Pubkey_prev_op = '76' + 'a9' + size_receiver_1 + output_script + '88' + 'ac'
    script_len_1 = hex(int(len(script_Pubkey_prev_op)/2))[2:]
    return script_len_1 + script_Pubkey_prev_op

def tx_outputs(receiver, value):
    if receiver[0] == 'm' or receiver[0] == 'n': 
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        size_receiver = hex(int(len(receiver_address)/2))[2:]
        locking_Script = '76' + 'a9' + size_receiver + receiver_address + '88' + 'ac'
        script_len_2 = hex(int(len(locking_Script)/2))[2:]
        return value_in_hex_NYB + script_len_2 + locking_Script
    
    elif receiver[0] == '2':
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '87'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script
    
def generate_raw_transaction(previous_tx_hash, previous_output_index, 
                             previous_output_script, number_of_outputs, 
                             receivers_btc, output_text = None):
    version = 1
    version = bytewise_reverse('{:08d}'.format(int(version)))

    utxo_ip = '01'
    prev_tx_datas = utxo_ip + tx_inputs_tx_id(previous_tx_hash, previous_output_index, previous_output_script)

    tx_output = ''
    for i in range(0, int(number_of_outputs)):
        receiver, value = receivers_btc[i]
        if output_text is not None and output_text[0]-1 == i:
            tx_output = op_return(output_text[1]) + tx_output
        else: 
            tx_output = tx_outputs(receiver, value) + tx_output
    next_output_data = '0' + str(number_of_outputs) + tx_output

    locktime = '00000000'
    SigHashCode = '01000000'


    raw_transaction = version + prev_tx_datas + next_output_data + locktime + SigHashCode
    return raw_transaction  

def bytes_padding(temp_no):
    if len(temp_no)/2 != 0:
        return '0' + temp_no
    else:
        return temp_no
        
def op_return(data):
    value_return = '0'*16
    len_data = hex(int(len(data)/2))[2:]
    partial_data =  '6a'+ bytes_padding(len_data) + data
    return value_return + bytes_padding(hex(int(len(partial_data)/2))[2:]) + partial_data

def sign_raw_transaction(tx, private_key):
    temp_bin = binascii.unhexlify(tx)
    temp_hash = hashlib.sha256(temp_bin).digest()
    hash2 = hashlib.sha256(temp_hash).hexdigest()
    tx_hash = hashlib.sha256(temp_hash).digest()
    signingKey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingKey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)
    SIG = str(binascii.hexlify(SIG),'ascii')
    return SIG

def decode_digital_signature(SIG, public_key):
    #Variables:
    header = SIG[0:2]
    Sig_Length = SIG[2:4]
    r_integer = SIG[4:6]
    r_length = SIG[6:8]
    r = SIG[8:8 + 2 * int(r_length,16)]
    s_integer = SIG[8 + 2 * int(r_length,16): 10 + 2 * int(r_length,16)]
    s_length = SIG[10 + 2 * int(r_length,16): 12 + 2 * int(r_length,16)]
    s = SIG[12 + 2 * int(r_length,16):]

     #_____DER_SIG_SCRIPT_ENCODER____________
    def pushdata(len_r, len_s):
        if len_r == '20' and len_s == '20':
            return str(47)    
        elif len_r == '20' or len_s == '21':
            return str(48)
        elif len_r == '21' or len_s == '20':
            return str(48)
        else:
            return str(49)

    def pubkey_opcode(a):
        return hex(int(int(len(a))/2))[2:]

    pushdata_opcode = pushdata(r_length, s_length)
    sighash_code = '01'
    pubkey_pushdata_opcode = pubkey_opcode(public_key)
    final_SigScript = pushdata_opcode + header + Sig_Length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + pubkey_pushdata_opcode + public_key
    return pubkey_opcode(final_SigScript) + final_SigScript

def finalize(tx, sig_script, output_script):
    print(output_script)
    first = tx[:-8]
    idx = first.find(output_script)
    idx2 = idx + len(output_script)
    print(first[:idx-2])
    print(sig_script)
    print(first[idx2:])
    print()
    second = f"{first[:idx-2]}{sig_script}{first[idx2:]}"
    return second

def sign(tx, address_data, output_script_full):
    SIG = sign_raw_transaction(tx=tx, private_key=address_data[0])
    sig_script = decode_digital_signature(SIG=SIG, public_key=address_data[1])
    ready = finalize(tx=tx, sig_script=sig_script, output_script=output_script_full)
    return ready

In [3]:
issued = 0.0266188 
first_tx_sum = 0.00002
second_tx_sum = 0.01
third_tx_sum = 0.005

tx_1 = generate_raw_transaction(previous_tx_hash=tx,
                                previous_output_index=output_idx,
                                previous_output_script = output_script,
                                number_of_outputs= 2,
                                output_text = (2, my_name_double_hashed[:8]),
                                receivers_btc=[
                                              (address_data_2[-1], first_tx_sum),
                                              (address_data_3[-1], 0)
                                ]
                               )

NameError: name 'address_data_2' is not defined

In [22]:
print('tx_1: \n' + tx_1)

tx_1: 
0100000001b0d9280cb0e571218460e79319335a524684bc9179eb5ab6439207ac2b811405010000001976a914027db649a3b298bca1c6de79f5ba590e72fd40ac88acffffffff020000000000000000066a04f2f27716d0070000000000001976a914680d76b17e681d21388aa6bcd04d91eeb523b89488ac0000000001000000


In [23]:
tx_1_ready = sign(tx_1, address_data_1, output_script_full)

76a914027db649a3b298bca1c6de79f5ba590e72fd40ac88ac
0100000001b0d9280cb0e571218460e79319335a524684bc9179eb5ab6439207ac2b81140501000000
8a47304402205cd3156ef98982759f356d4ac5bd0af70aa2f04f1602de2f092681d36ca806bb0220185851e909d7004e6a568cbc3a2b4fd32790156875dac3fabb2b906d5e084320014104a92d69cdeedc1d359ce623fc2f5240a341882b0cc0d2cc03804fcebce2ecf3ae8d9b125cfe4f4eb79bf94b94c264e95066db9b103c4418c9860f3131f2f1c1c3
ffffffff020000000000000000066a04f2f27716d0070000000000001976a914680d76b17e681d21388aa6bcd04d91eeb523b89488ac00000000



In [24]:
print('tx_1_ready: \n' + tx_1_ready)

tx_1_ready: 
0100000001b0d9280cb0e571218460e79319335a524684bc9179eb5ab6439207ac2b811405010000008a47304402205cd3156ef98982759f356d4ac5bd0af70aa2f04f1602de2f092681d36ca806bb0220185851e909d7004e6a568cbc3a2b4fd32790156875dac3fabb2b906d5e084320014104a92d69cdeedc1d359ce623fc2f5240a341882b0cc0d2cc03804fcebce2ecf3ae8d9b125cfe4f4eb79bf94b94c264e95066db9b103c4418c9860f3131f2f1c1c3ffffffff020000000000000000066a04f2f27716d0070000000000001976a914680d76b17e681d21388aa6bcd04d91eeb523b89488ac00000000


https://blockstream.info/testnet/tx/3954b3b61a05356f51ca1577fba11b669618bbd08dcc03cbb8d415b01d1ef4fd
Осуществленная тразакция:  
https://live.blockcypher.com/btc-testnet/tx/3954b3b61a05356f51ca1577fba11b669618bbd08dcc03cbb8d415b01d1ef4fd/

## Task 4 
From address_2, Send some bitcoins to your 2-of-3 MultiSignature Address.

In [25]:
class variableInteger:
    def reverse_string_in_pair(a_str_ip):
        i = 0
        rev_a = []
        while i != len(a_str_ip):
            temp_value = a_str_ip[i] + a_str_ip[i+1]
            rev_a.append(temp_value)
            i = i + 2
        rev_a.reverse()
        return ''.join(rev_a)
    
    def padding(b):
        if int(len(b) % 2) == 0:
            return b
        else:
            return '0' + b
    
    def variable_integer(a):
        if type(a) == int:
            temp = a
        else:
            temp = int(len(a)/2)
        
        if temp <= int('0xfc',16):
            return variableInteger.padding(hex(temp)[2:])
        elif temp > int('0xfc',16) and temp <= int('0xffff',16):
            return 'fd' +  variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffff',16) and temp <= int('0xffffffff',16):
            return 'fe' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffffffff',16) and temp <= int('0xffffffffffffffff',16):
            return 'ff' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        
def multisig_digital_signature(raw_tx, private_key):
    unsigned = raw_tx
    
    temp_bin = binascii.unhexlify(unsigned)
    temp_hash = hashlib.sha256(temp_bin).digest()
    tx_hash = hashlib.sha256(temp_hash).digest()
    signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)   
    temp_sig = str(binascii.hexlify(SIG),'ascii')
    return temp_sig     

def multisig_reverse_string_in_pair(a_str_ip):
    i = 0
    rev_a = []
    while i != len(a_str_ip):
        temp_value = a_str_ip[i] + a_str_ip[i+1]
        rev_a.append(temp_value)
        i = i + 2
    rev_a.reverse()
    return ''.join(rev_a)

def multisig_bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = reverse_string_in_pair(str(temp_1).zfill(16))
    return temp_2

def multisig_tx_inputs_tx_id(previous_tx_hash, previous_output_index):
    previous_tx_hash_reversed = reverse_string_in_pair(previous_tx_hash)
    previous_output_index = reverse_string_in_pair('{:08d}'.format(int(previous_output_index)))
    return previous_tx_hash_reversed + previous_output_index

def multisig_redeem_place_holder(redeemScript):
    return variableInteger.variable_integer(redeemScript) + redeemScript

def multisig_tx_outputs(receiver, value):
    if receiver[0] == 'm' or receiver[0] == 'n': 
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = '76' + 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '88' + 'ac'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script
    
    elif receiver[0] == '2':
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '87'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script

In [26]:
def mk_multisig_tx(previous_tx_hash, previous_output_index, redeemScript, address_data, value, pr_keys):
    sig_loop = 2
    version = '01000000'
    input_ms = '01' + multisig_tx_inputs_tx_id(previous_tx_hash=previous_tx_hash, 
                                             previous_output_index = previous_output_index)
    redeem_Script = multisig_redeem_place_holder(redeemScript=redeemScript)
    tx_input_value = input_ms + redeem_Script + 'f'*8
    tx_output = ''
    tx_output = multisig_tx_outputs(receiver = address_data[-1], value=value)
    next_output_data =  variableInteger.variable_integer(1) + tx_output

    locktime = '00000000'
    SigHashCode = '01000000'
    tx_to_sign = version + tx_input_value + next_output_data + locktime + SigHashCode
    print("Raw Transaction to Sign:")
    print(tx_to_sign)

    signatures = ''
    for i in range(2):
        signature_temp =  multisig_digital_signature(tx_to_sign, private_key=pr_keys[i]) + '01'
        signature_temp_2 = variableInteger.variable_integer(signature_temp) + signature_temp
        signatures = signatures + signature_temp_2

    SigScript_places = '00' + signatures + '4c' + redeem_Script 
    SigScript =  variableInteger.variable_integer(SigScript_places) + SigScript_places
    print("\nSigScript:", SigScript)

    Broadcasting_transaction = version + input_ms + SigScript + 'f'*8 + next_output_data + locktime
    print("Transaction for Broadcast:")
    print(Broadcasting_transaction)

In [27]:
tx1_hash = "3954b3b61a05356f51ca1577fba11b669618bbd08dcc03cbb8d415b01d1ef4fd"
output_idx_1 = 1
output_script_1 = "680d76b17e681d21388aa6bcd04d91eeb523b894"
output_script_1_full= "76a914680d76b17e681d21388aa6bcd04d91eeb523b89488ac"

In [28]:
tx_2 = generate_raw_transaction(previous_tx_hash=tx1_hash,
                                previous_output_index=output_idx_1,
                                previous_output_script = output_script_1,
                                number_of_outputs= 1,
                                receivers_btc=[(multisig_adress, 0.00001)]
                               )

In [29]:
tx_2_ready = sign(tx_2, address_data_2, output_script_1_full)

76a914680d76b17e681d21388aa6bcd04d91eeb523b89488ac
0100000001fdf41e1db015d4b8cb03cc8dd0bb1896661ba1fb7715ca516f35051ab6b3543901000000
8a473044022009399170c9a886723f1adaafbecccc11b5fecf98405f9e0b29052d1461fe86d70220299fe8667265684d01405288313252626ec7fe997ac19e54391c06a91a286143014104a5aa6ac17359ebeeb3843c24ffca8b105f258c8fc3183a6002d462456c2460123c8ea3964c0452972fb3988de951729b5e3a0613d79df5bdf825d024233b8738
ffffffff01e8030000000000001976a914027db649a3b298bca1c6de79f5ba590e72fd40ac88ac00000000



In [30]:
print('tx_2 is ready: \n' + tx_2_ready)

tx_2 is ready: 
0100000001fdf41e1db015d4b8cb03cc8dd0bb1896661ba1fb7715ca516f35051ab6b35439010000008a473044022009399170c9a886723f1adaafbecccc11b5fecf98405f9e0b29052d1461fe86d70220299fe8667265684d01405288313252626ec7fe997ac19e54391c06a91a286143014104a5aa6ac17359ebeeb3843c24ffca8b105f258c8fc3183a6002d462456c2460123c8ea3964c0452972fb3988de951729b5e3a0613d79df5bdf825d024233b8738ffffffff01e8030000000000001976a914027db649a3b298bca1c6de79f5ba590e72fd40ac88ac00000000


https://blockstream.info/testnet/tx/aaad7dfcecaf3dc87f4991a68c8be6a7303b09ba0549aeb756753cdf77275a85
Осуществленная транзакция:  
https://live.blockcypher.com/btc-testnet/tx/aaad7dfcecaf3dc87f4991a68c8be6a7303b09ba0549aeb756753cdf77275a85/

## Task 5 (4 points) 
From 2-of-3 MultiSignature Address. Send some bitcoins back to address_1.



In [31]:
def reverse_string_in_pair(a_str_ip):
        i = 0
        rev_a = []
        while i != len(a_str_ip):
            temp_value = a_str_ip[i] + a_str_ip[i+1]
            rev_a.append(temp_value)
            i = i + 2
        rev_a.reverse()
        return ''.join(rev_a)

In [32]:
tx2_hash = "aaad7dfcecaf3dc87f4991a68c8be6a7303b09ba0549aeb756753cdf77275a85"
output_idx_2 = 1
output_script_2 = "027db649a3b298bca1c6de79f5ba590e72fd40ac"
output_script_2_full= "76a914027db649a3b298bca1c6de79f5ba590e72fd40ac88ac"

Взял код из семинара 1 в 1, но оно почему-то все равно не завелось нормально, хотя вроде бы все правильно принтит

In [33]:
mk_multisig_tx(previous_tx_hash= tx2_hash,
               previous_output_index = output_idx_2,
                redeemScript=multisig_redeem_script,
               address_data=address_data_1, 
               value=third_tx_sum,
               pr_keys = [address_data_1[0], address_data_2[0]] 
)

Raw Transaction to Sign:
0100000001855a2777df3c7556b7ae4905ba093b30a7e68b8ca691497fc83dafecfc7dadaa0100000069522103a92d69cdeedc1d359ce623fc2f5240a341882b0cc0d2cc03804fcebce2ecf3ae2102a5aa6ac17359ebeeb3843c24ffca8b105f258c8fc3183a6002d462456c2460122102a36f384e5625081d055e0c8f202571abb482ce8bc7369687857a1266cbcaf46253aeffffffff0120a10700000000001976a914027db649a3b298bca1c6de79f5ba590e72fd40ac88ac0000000001000000

SigScript: fdfd00483045022100c533740164ceef5c1b7eed131cc74fe46d0d03a698a07a2d1ff960b13a97da3902207d4197a19857a242c9bcca32079c815883c61bf42d5d9c29a9725bb73dbe7cd701473044022040c3b6829d8ee143a25a43d05287fae68492e9e8556e82a3ea1c3c505951b4df02204ad755f43654040e3bd90d981fd22baf51a41c0d7e5304a41b95ab03df1004cc014c69522103a92d69cdeedc1d359ce623fc2f5240a341882b0cc0d2cc03804fcebce2ecf3ae2102a5aa6ac17359ebeeb3843c24ffca8b105f258c8fc3183a6002d462456c2460122102a36f384e5625081d055e0c8f202571abb482ce8bc7369687857a1266cbcaf46253ae
Transaction for Broadcast:
0100000001855a2777df3c7556b7ae4905ba